In [ ]:
!gdown --id 1ay5DcH64Qao1HR7CQnR6Cl1hbBMgGqXj -O DOE_High_School_Directory_2014-2015.csv
!gdown --id 13BozEl3JtS43Xuu2Ek9IwMULpWjPH4VC -O SAT_Results.csv

Downloading...
From: https://drive.google.com/uc?id=1ay5DcH64Qao1HR7CQnR6Cl1hbBMgGqXj
To: /content/DOE_High_School_Directory_2014-2015.csv
100% 1.30M/1.30M [00:00<00:00, 20.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=13BozEl3JtS43Xuu2Ek9IwMULpWjPH4VC
To: /content/SAT_Results.csv
100% 28.3k/28.3k [00:00<00:00, 4.14MB/s]


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 8.3 kB/s 
     |████████████████████████████████| 198 kB 44.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=94b0bb3ba07f96b69c28b5a3a1b4f9f28627bb9fc74324bf7874a509c06e94ad
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pyspark
import csv
sc = pyspark.SparkContext()

ValueError: ignored

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
def findDuplicateSequence(s):
  prev = ""
  tmp = ""
  for i in s:
    if prev == i:
      tmp = tmp+i
    else:
      if len(tmp) > 1:
        yield tmp
      prev = i
      tmp = i

  if len(tmp) > 1:
    yield tmp


next(findDuplicateSequence(iter('aabccccedaaa'))) # 'aa'
#list(findDuplicateSequence(iter('aabccccedaaa'))) # ['aa', 'cccc', 'aaa']

'aa'

In [ ]:
import functools
countWord = lambda text: functools.reduce(lambda a,b: a+len(b.split[' ']), text, 0)

countWord(['Hello there']) # 2
#countWord(['Welcome to the big data exam']) # 6
#countWord(['Hello there', 'Welcome to the big data exam']) # 8

TypeError: ignored

In [ ]:
def everyOtherRecord(reader):
  for line in reader:
    yield line
    next(reader)
    

file = open('SAT_Results.csv')
reader = csv.reader(file)

for line in everyOtherRecord(reader):
  print(line)

['DBN', 'SCHOOL NAME', 'Num of SAT Test Takers', 'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score', 'SAT Writing Avg. Score']
['21K410', 'ABRAHAM LINCOLN HIGH SCHOOL', '475', '396', '437', '393']
['30Q301', 'ACADEMY FOR CAREERS IN TELEVISION AND FILM', '98', '410', '440', '405']
['18K637', 'ACADEMY FOR CONSERVATION AND THE ENVIRONMENT', '35', '363', '381', '367']
['17K751', 'ACADEMY FOR HEALTH CAREERS', 's', 's', 's', 's']
['11X270', 'ACADEMY FOR SCHOLARSHIP AND ENTREPRENEURSHIP: A COLLEGE BOARD SCHOOL', '56', '386', '394', '361']
['14K404', 'ACADEMY FOR YOUNG WRITERS', '68', '387', '357', '381']
['13K336', 'ACADEMY OF BUSINESS AND COMMUNITY DEVELOPMENT', '9', '439', '374', '418']
['24Q264', 'ACADEMY OF FINANCE AND ENTERPRISE', '89', '405', '454', '421']
['19K618', 'ACADEMY OF INNOVATIVE TECHNOLOGY', '60', '371', '371', '354']
['32K552', 'ACADEMY OF URBAN PLANNING', '67', '342', '364', '354']
['07X600', 'ALFRED E. SMITH CAREER AND TECHNICAL EDUCATION HIGH SCHOOL', '76', '394', '

RuntimeError: ignored

In [ ]:
# We're creating two RDD, one is from the sample README file
# and the other is directly from a list within the notebook.

rdd = sc.textFile('sample_data/README.md')
testRdd = sc.parallelize([1,2,3,4,5])

In [ ]:
# Here, we're showing the difference between map() and flatMap()
# doing the line split and get back the first 3 elements, take(3).
# - map() is a one-to-one mapping, just like Python, so the first
# line prints out 3 lists, each consists words per each.
# - flatMap() is a one-to-many mapping, like MapReduce's map(). So
# the second line prints out only 3 words.

wordsPerLine = rdd.map(lambda line: line.split()).take(3)
words = rdd.flatMap(lambda line: line.split()).take(3)

print('%s\n\n%s' % (wordsPerLine, words))

[['This', 'directory', 'includes', 'a', 'few', 'sample', 'datasets', 'to', 'get', 'you', 'started.'], [], ['*', '`california_housing_data*.csv`', 'is', 'California', 'housing', 'data', 'from', 'the', '1990', 'US']]

['This', 'directory', 'includes']


In [ ]:
# This is the word count example with Spark using the approach
# shown in the slides, i.e. staying true to the MapReduce paradigm.
# Note that groupByKey() will sort and group everything together by
# keys first. Then the function in mapValues() will each get applied
# per each (key, list of values) pair. This could be an issue if we
# have a pair with lots of values since all of the values have to be
# stored in memory.

wc = rdd.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .groupByKey() \
        .mapValues(lambda values: sum(values))
wc.take(2)

[('this', 1), ('directory', 1)]

In [ ]:
# This is another approach with reduceByKey() instead of groupByKey().
# The reduce function provided for reduceByKey() only takes 2 params
# at a time, thus, doesn't suffer the scalability issue. It also has
# better benefits in term of parallelism.

wc = rdd.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .reduceByKey(lambda x,y: x+y)
wc.take(2)

[('this', 1), ('directory', 1)]

In [ ]:
# If we'd like to compute the top 3 most popular words in Spark. We
# can use the RDD's top() function directly. This is much easier
# than the two-step MapReduce job, where we had to first compute the
# top 3 words per partition, then another top 3 on top of that. In
# fact, this is exactly how Spark RDD's top() function is implemented.
# More info can be found here:
# https://github.com/apache/spark/blob/master/python/pyspark/rdd.py#L1459
wc.top(3, key=lambda x: x[1])

[('is', 4), ('*', 3), ('a', 3)]

## LAB 4 - Task 1

In [ ]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [ ]:
# We read the SAT score to our RDD. Note that the use_unicode can be
# changed accordingly to your data file to handle Unicode. If you cannot
# parse your data due to an 'utf8' or 'ascii' decoding issue, it might
# be a good thing to try flipping the use_unicode parameter here.

sat = sc.textFile(SAT_FN, use_unicode=True).cache()

# This line for us to list the column index and column names to see
# which column we need to use for our task. In this case, we're
# interested in the number of test takers (#2) and the math score (#4).
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [ ]:
# Note that, our data input includes a header line that we don't want to
# use in analysis. We can remove the header line from our RDD by doing
# a 'filter' to remove all rows that matches the header like below. Though
# this works, it means that we have to apply the filter function on *all*
# row, which could be a lot of computation.

noHeaderRDD = sat.filter(lambda x: not x.startswith('DBN,SCHOOL'))
print(sat.first())
print(noHeaderRDD.first())

DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387


In [ ]:
# Alternatively, we can perform the header checking per-partition, instead
# of per-row like below. mapPartitions() is another type of map operators
# in Spark that is similar to Hadoop Streaming's map(). It is many-to-many.
# RDD in Spark are divided into partitions (as we read or as provided by
# HDFS), each partition can be processed in parallel using a function
# supplied to the mapPartitions() call.
# 
# In addition to mapPartitions(), Spark also provides a variation called
# mapPartitionsWithIndex() that provides information on which partition
# we are currently processing. Indeed, mapPartitionsWithIndex() is the
# the operator with the lowest overhead (since mapPartitions() get mapped
# to mapPartitionsWithIndex) and also the most efficient one among all the
# map operators.
#
# So our logic below is to use the partition index to check if we're hitting
# the header (aka the first partition). If so, we just skip the first row.

def extractScores(partId, records):
    if partId==0:
        next(records)
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[2]!='s': # to filter our bad-quality data
            (dbn,takers,score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(5)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35))]

In [ ]:
# Here we do the same thing with the school directory data
schools = sc.textFile(HSD_FN, use_unicode=True).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [ ]:
def extractSchools(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500: # filter to keep the large schools
                yield (dbn, boro)

largeSchools = schools.mapPartitionsWithIndex(extractSchools)

In [ ]:
scores = largeSchools.join(satScores).values() \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1]))

In [ ]:
scores.collect()

[('Bronx', 470),
 ('Staten Island', 477),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Queens', 474)]

## LAB 4 - Task 1 -- Using DataFrame

In [ ]:
# Spark's DataFrame is part of Spark SQL, thus, we need to import
# the appropriate package in order to use Spark's DataFrame and SQL
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [ ]:
dfSchools = spark.read.load(HSD_FN, format='csv', header=True, inferSchema=True)
dfSchools = dfSchools.na.drop(subset=['boro'])
dfSchools = dfSchools.filter(dfSchools['total_students']>500)
dfSchools = dfSchools.select('dbn', 'boro')
dfSchools.count()

176

In [ ]:
dfScores = spark.read.load(SAT_FN, format='csv', header=True, inferSchema=True)
dfScores = dfScores.select(
    'DBN',
    dfScores['`SAT Math Avg. Score`'].cast('int').alias('score'),
    dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers')
).na.drop()
dfScores1 = dfScores.select(
    'DBN',
    (dfScores.score*dfScores.ntakers).alias('sum'),
    'ntakers')
dfScores1.head()

Row(DBN='02M047', sum=6400, ntakers=16)

In [ ]:
dfResults = dfSchools.join(dfScores1, dfSchools.dbn==dfScores1.DBN, how='inner')
dfResults = dfResults.groupBy('boro').sum('sum', 'ntakers')
dfResults = dfResults.withColumn('avg', (dfResults[1]/dfResults[2]).cast('int'))
dfResults = dfResults.select('boro', 'avg')
dfResults.show()

+-------------+---+
|         boro|avg|
+-------------+---+
|       Queens|474|
|     Brooklyn|487|
|Staten Island|477|
|    Manhattan|514|
|        Bronx|470|
+-------------+---+



## LAB 4 - Task 2